In [6]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [7]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 4475.99 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 4.96 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [8]:
path = "C:\\Users\\sefi\\PPSN\\PPSN\\Data Sets\\Original\\50 D\\Training_Data_Sets\\train_2_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_200Samples.csv')).iloc[:,-1]
path = "C:\\Users\\sefi\\PPSN\\PPSN\\Data Sets\\Latent\\90 %\\50 D\\Training_Data_Sets\\latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv(path[:-33]+str('Test_Data_Sets\\latent_50D.csv'), index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [9]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,6):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 5000)

n_obs , dim =  100, 5
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([100,15])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=424970, Wed Apr  8 19:41:17 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [55549696.62883809] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.76e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
    1      8 5.554969662883809e+07 1.0e+00 2.76e+00  3e+00  3e+00 0:00.0
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.76e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the f

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=448230, Wed Apr  8 19:41:18 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [1.76743436e+09] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 3.725512290446985e+08 1.0e+00 2.28e+00  2e+00  2e+00 0:00.0
WARNING ():  function_values is not a list of scalars,
                        the first element equals [1.47115434e+09] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
    2     16 8.074042799764901e+07 1.2e+00 2.04e+00  2e+00  2e+00 0:00.0
    3     24 1.558413773052113e+08 1.4e+00 1.73e+00  2e+00  2e+00 0:00.0
   13    104 6.944088586918461e+07 3.7e+00 2.18e+00  2e+00  3

WARNING (iteration=11):  flat fitness (f=55549696.628838, sigma=1.94e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=11):  flat fitness (f=55549696.628838, sigma=1.94e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=11):  flat fitness (f=55549696.628838, sigma=1.94e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=12):  flat fitness (f=55549696.628838, sigma=1.89e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborate

    1      8 5.211812922654726e+07 1.0e+00 2.46e+00  2e+00  3e+00 0:00.0
WARNING ():  function_values is not a list of scalars,
                        the first element equals [52118129.22654726] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
WARNING (iteration=2):  flat fitness (f=52118129.226547, sigma=2.55e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
    2     16 5.211812922654726e+07 1.3e+00 2.55e+00  2e+00  3e+00 0:00.0
WARNING (iteration=2):  flat fitness (f=52118129.226547, sigma=2.55e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=2):  flat fitness (f=52118129.226547, sigma=2.55e+00)

Run : 5
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=478129, Wed Apr  8 19:41:21 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [55549696.62883809] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.78e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
    1      8 5.554969662883809e+07 1.0e+00 2.78e+00  3e+00  3e+00 0:00.0
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.78e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the f

...

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=437460, Wed Apr  8 19:42:33 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [52118129.22654726] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 5.211812922653511e+07 1.0e+00 2.43e+00  2e+00  3e+00 0:00.0
WARNING ():  function_values is not a list of scalars,
                        the first element equals [52118129.22654726] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
    2     16 5.211812915037329e+07 1.2e+00 2.29e+00  2e+00  2e+00 0:00.0
    3     24 5.211812922653527e+07 1.2e+00 2.12e+00  2e+00  2e+00 0:00.0
   13    104 5.211747165013021e+07 1.8e+00 9.13e-01  6e

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=542889, Wed Apr  8 19:42:35 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [55549696.62883809] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.30e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
    1      8 5.554969662883809e+07 1.0e+00 2.30e+00  2e+00  2e+00 0:00.0
WARNING (iteration=1):  flat fitness (f=55549696.628838, sigma=2.30e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness m

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=446945, Wed Apr  8 19:42:36 2020)
WARNING ():  function_values is not a list of scalars,
                        the first element equals [1.62275976e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.622759759307934e+08 1.0e+00 2.59e+00  2e+00  3e+00 0:00.0
WARNING ():  function_values is not a list of scalars,
                        the first element equals [7.70902432e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed)
    2     16 4.413165257960303e+08 1.3e+00 2.62e+00  2e+00  3e+00 0:00.0
    3     24 3.319645952487960e+08 1.5e+00 2.44e+00  2e+00  3e+00 0:00.0
   13    104 4.185742958211023e+07 3.9e+00 1.11e+00  8e-01  1

WARNING (iteration=11):  flat fitness (f=52118129.226547, sigma=1.77e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=12):  flat fitness (f=52118129.226547, sigma=1.88e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=12):  flat fitness (f=52118129.226547, sigma=1.88e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately.
WARNING (iteration=12):  flat fitness (f=52118129.226547, sigma=1.88e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborate

   13    104 3.882013715225837e+07 2.9e+00 1.14e+00  8e-01  1e+00 0:00.1
termination on maxfevals=100 (Wed Apr  8 19:42:38 2020)
final/bestever f-value = 1.990476e+07 1.990476e+07
incumbent solution: [3.618668810289191, 4.982048863691166, 0.28951572652344293, -0.18475364196534472, -1.558327108940567]
std deviation: [0.9444218539785899, 1.1734024565499934, 0.7941934327086945, 1.2574974832090824, 0.7969847214278313]


In [15]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values.csv')
Krig_Fun = np.zeros(100)
SVM_Fun = np.zeros(100)
KNN_Fun = np.zeros(100)
ELN_Fun = np.zeros(100)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:5])
    SVM_Fun [i] = F2(X_Values.iloc[i,5:10])
    ELN_Fun [i] = F2(X_Values.iloc[i,10:15])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([82726.38832564]), count=array([1])) 17784878.918727104
SVM
ModeResult(mode=array([442851.41339762]), count=array([1])) 11890095.698423877
ELN
ModeResult(mode=array([222289.8564792]), count=array([1])) 4998631.672587333
